In [1]:
import pandas as pd 
import numpy as np 
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import TfidfVectorizer

# Precisamos fazer isso para importar variávies de outro notebook.
%store -r credits
%store -r movies

C:\Users\renan\AppData\Roaming\Python\Python312\site-packages\IPython\extensions\storemagic.py:148: UserWarning: This is now an optional IPython functionality, using autorestore/credits requires you to install the `pickleshare` library.
  obj = db["autorestore/" + arg]
C:\Users\renan\AppData\Roaming\Python\Python312\site-packages\IPython\extensions\storemagic.py:148: UserWarning: This is now an optional IPython functionality, using autorestore/movies requires you to install the `pickleshare` library.
  obj = db["autorestore/" + arg]


In [2]:
df1=pd.read_csv(r"tmdb_5000_credits.csv", encoding='utf-8', sep=',')
df2=pd.read_csv(r"tmdb_5000_movies.csv", encoding='utf-8', sep=',')

df1.columns = ['id','title','cast','crew']
dados = df2.merge(df1,on='id')

dados.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title_x,vote_average,vote_count,title_y,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...",...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...",...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]",...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


# Content Based Filtering

Neste sistema de recomendação, o conteúdo do filme (sinopse, elenco, equipe, palavra-chave, slogan etc.) é usado para encontrar sua semelhança com outros filmes. Em seguida, os filmes que são mais provavelmente semelhantes são recomendados.

In [3]:
dados['overview'].head(5)

0    In the 22nd century, a paraplegic Marine is di...
1    Captain Barbossa, long believed to be dead, ha...
2    A cryptic message from Bond’s past sends him o...
3    Following the death of District Attorney Harve...
4    John Carter is a war-weary, former military ca...
Name: overview, dtype: object

In [14]:
# Inicializando o objeto Vetorizador TF-IDF com a remoção de stopwords em inglês
vetorizador_tfidf = TfidfVectorizer(stop_words='english')

# Preenchendo valores nulos na coluna 'visão geral' com uma string vazia
dados['overview'] = dados['overview'].fillna('')

# Criando uma matriz TF-IDF a partir dos textos da coluna 'visão_geral'
# matriz_tfidf = vetorizador_tfidf.fit_transform(dados['overview'])

# Vetorizando a coluna 'overview'
overview_tfidf = vetorizador_tfidf.fit_transform(dados['overview']).toarray()

# Vetorizando a coluna 'title'
title_tfidf = vetorizador_tfidf.fit_transform(dados['title_x']).toarray()

# Combinando as matrizes TF-IDF
combined_tfidf = np.hstack((title_tfidf, overview_tfidf))


# Extraindo a forma (shape) da matriz TF-IDF
# Essa linha retorna uma tupla representando o número de linhas (documentos) e colunas (features) na matriz TF-IDF
print("Forma da Matriz TF-IDF:", combined_tfidf.shape)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
Forma da Matriz TF-IDF: (4803, 25472)


In [15]:
# Calculando a similaridade cosseno entre as linhas da matriz TF-IDF
similaridade_cosseno = linear_kernel(combined_tfidf, combined_tfidf)
print(similaridade_cosseno)

# Criando uma série de índices onde os índices são os títulos dos documentos e os valores são os índices correspondentes no DataFrame
indices = pd.Series(dados.index, index=dados['title_x']).drop_duplicates()

[[2.         0.         0.         ... 0.         0.         0.        ]
 [0.         2.         0.         ... 0.02160533 0.         0.        ]
 [0.         0.         2.         ... 0.01488159 0.         0.        ]
 ...
 [0.         0.02160533 0.01488159 ... 2.         0.01609091 0.00701914]
 [0.         0.         0.         ... 0.01609091 2.         0.01171696]
 [0.         0.         0.         ... 0.00701914 0.01171696 2.        ]]


In [39]:
def obter_recomendacoes(titulo, similaridade_cosseno=similaridade_cosseno):

    if titulo not in indices:
        return "Filme não encontrado."
    indice = indices[titulo]
    scores_similares = list(enumerate(similaridade_cosseno[indice]))
    scores_similares = sorted(scores_similares, key=lambda x: x[1], reverse=True)

    # Faz com que mostrem filmes que tenham um score de similaridade acima de 0.6
    scores_similares = [score for score in scores_similares[1:] if score[1] > 0.6]
    indices_filmes_recomendados = [i[0] for i in scores_similares]

      # Se nenhum filme atender ao critério, você pode optar por retornar uma mensagem ou lista vazia
    if not indices_filmes_recomendados:
        return "Nenhum filme similar encontrado."
    
    # Certifique-se de que você está acessando os nomes de colunas corretos
    recomendacoes = dados.loc[indices_filmes_recomendados, ['title_x', 'overview']]
    recomendacoes['cosine_similarity'] = [s[1] for s in scores_similares]  # adicionando valores de similaridade cosseno
    
    return recomendacoes


In [41]:
obter_recomendacoes('The Fast and the Furious')

,title_x,overview,cosine_similarity
500,2 Fast 2 Furious,It's a major double-cross when former police o...,1.086473
204,Fast Five,Former cop Brian O'Conner partners with ex-con...,1.042821
44,Furious 7,Deckard Shaw seeks revenge against Dominic Tor...,0.954234
405,The Fast and the Furious: Tokyo Drift,"In order to avoid a jail sentence, Sean Boswel...",0.680311


In [42]:
obter_recomendacoes('The Expendables') # Os Mercenários

,title_x,overview,cosine_similarity
307,The Expendables 3,"Barney, Christmas and the rest of the team com...",1.183428
290,The Expendables 2,Mr. Church reunites the Expendables for what s...,1.061102


Podemos notar que quando o filme possui em sua descrição os mesmos personagens e títulos parecidos, a precisão do cosseno é superior a 1.

In [43]:
obter_recomendacoes('The Terminator')

,title_x,overview,cosine_similarity
279,Terminator 2: Judgment Day,Nearly 10 years have passed since Sarah Connor...,0.920224
43,Terminator Salvation,"All grown up in post-apocalyptic 2018, John Co...",0.844573
108,Terminator Genisys,"The year is 2029. John Connor, leader of the r...",0.747815
93,Terminator 3: Rise of the Machines,It's been 10 years since John Connor saved Ear...,0.705550


Podemos notar que quando os personagens mudam, o valor de cosseno apesar de ser próximo a 1, acabam obtendo um resultado menor.

In [37]:
obter_recomendacoes('Star Wars')

,title_x,overview,cosine_similarity
3208,Star Wars: Clone Wars: Volume 1,"The Saga continues with the Emmy-winning ""Star...",0.806029
233,Star Wars: Episode I - The Phantom Menace,"Anakin Skywalker, a young slave strong with th...",0.636956
229,Star Wars: Episode III - Revenge of the Sith,"Years after the onset of the Clone Wars, the n...",0.615198


Percebemos no código acima, que vários filmes não aparecem, isso se da por que a funcão foi programada para calcular similaridade de filmes que possuem cosseno acima de 0.6
Neste caso, filmes como Star Trek, não irão aparecer.

Para verificar estes casos, criaremos uma nova função que tenha como parâmetro o limite de score desejado, com valor padrão de 0.6

In [44]:
# Se criarmos uma função, na qual definimos o valor de limite do score, poderemos visualizar filmes mais similares:

def recomend_score(titulo, similaridade_cosseno=similaridade_cosseno, limite_score=0.6):

    if titulo not in indices:
        return "Filme não encontrado."
    indice = indices[titulo]
    scores_similares = list(enumerate(similaridade_cosseno[indice]))
    scores_similares = sorted(scores_similares, key=lambda x: x[1], reverse=True)

    # Faz com que mostrem filmes que tenham um score de similaridade acima de 0.6
    scores_similares = [score for score in scores_similares[1:] if score[1] > limite_score]
    indices_filmes_recomendados = [i[0] for i in scores_similares]

      # Se nenhum filme atender ao critério, você pode optar por retornar uma mensagem ou lista vazia
    if not indices_filmes_recomendados:
        return "Nenhum filme similar encontrado."
    
    # Certifique-se de que você está acessando os nomes de colunas corretos
    recomendacoes = dados.loc[indices_filmes_recomendados, ['title_x', 'overview']]
    recomendacoes['cosine_similarity'] = [s[1] for s in scores_similares]  # adicionando valores de similaridade cosseno
    
    return recomendacoes

In [45]:
recomend_score('Star Wars', limite_score=0.3) # Desta forma, estamos calculando filmes com similidade maior que 0.3, trazendo mais resultados possíveis.

,title_x,overview,cosine_similarity
3208,Star Wars: Clone Wars: Volume 1,"The Saga continues with the Emmy-winning ""Star...",0.806029
233,Star Wars: Episode I - The Phantom Menace,"Anakin Skywalker, a young slave strong with th...",0.636956
229,Star Wars: Episode III - Revenge of the Sith,"Years after the onset of the Clone Wars, the n...",0.615198
230,Star Wars: Episode II - Attack of the Clones,"Ten years after the invasion of Naboo, the gal...",0.568819
1576,Bride Wars,Two best friends become rivals when their resp...,0.539843
1518,Dragon Wars: D-War,"Based on the Korean legend, unknown creatures ...",0.478197
56,Star Trek Beyond,The USS Enterprise crew explores the furthest ...,0.436048
158,Star Trek,The fate of the galaxy rests in the hands of b...,0.436048
1315,Rock Star,Rock Star tells the story of Chris Cole and a ...,0.424555
1990,The Empire Strikes Back,"The epic saga continues as Luke Skywalker, in ...",0.422981


Podemos observar, que os filmes com maior _Score_ fazem parte da saga <b>Star Wars</b>, porém filmes que possuem score menor do que 0.5, geralmente só possuem o título, ou _Overview_ parecidos com o filme indicado como parâmetro na função.

In [49]:
recomend_score('The Fast and the Furious', limite_score=0.01)

,title_x,overview,cosine_similarity
500,2 Fast 2 Furious,It's a major double-cross when former police o...,1.086473
204,Fast Five,Former cop Brian O'Conner partners with ex-con...,1.042821
44,Furious 7,Deckard Shaw seeks revenge against Dominic Tor...,0.954234
405,The Fast and the Furious: Tokyo Drift,"In order to avoid a jail sentence, Sean Boswel...",0.680311
3697,Fast Times at Ridgemont High,Follows a group of high school students growin...,0.343047
...,...,...,...
3886,It's a Wonderful Life,George Bailey has spent his entire life giving...,0.010122
4094,Alexander's Ragtime Band,"Roger Grant, a classical violinist, disappoint...",0.010098
4527,Twin Falls Idaho,Francis and Blake Falls are Siamese twins who ...,0.010045
1813,Gran Torino,Walt Kowalski is a widower who holds onto his ...,0.010031


Podemos notar que ao colocar um Score extremamente baixo, as recomendações podem nem fazer sentido, ou apenas alguns filmes tenham uma temática parecida, que usam palavras parecidas em suas Overview ou Títulos.

Um dos pontos positivos de utilizar TfidfVectorizer é que os filmes com maior qualidade em suas descrição, e maior similaridade em títulos, terão melhores resultados
Porém, quando sua descrição ou título forem vagos, o seu Score de similaridade tende a cair, porém é notável que quando o Score está entre 0.5 e 1, as similaridades são mais aceitáveis.
Em alguns casos, as similaridades abaixo de 0.5 podem mostrar filmes com temáticas parecidas, porém a partir de 0.3, o algoritmo considera somente palavras no título, não levando em consideração a Overview do Filme.